Connect to ElasticSearch Server

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print("Connected to ElasticSearch server!")
pprint(client_info.body)

Connected to ElasticSearch server!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'T1HeaWnRTOqX_BBgREVVbA',
 'name': '4cfd098368b9',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-10-21T10:06:21.288851013Z',
             'build_flavor': 'default',
             'build_hash': '25d88452371273dd27356c98598287b669a03eae',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.1',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.0'}}


1. Common types

1.1 Binary

In [2]:
# delete old data
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index", mappings={
    "properties": {
        "image_data": {
            "type": "binary" 
        }
    }
})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [3]:
import base64
image_path = "images/video_thumbnai_rag_chatbot.png"
with open(image_path, "rb") as image_file:
    image_bytes = image_file.read()
    image_base64 = base64.b64encode(image_bytes).decode('utf-8')
image_base64[:100]

'iVBORw0KGgoAAAANSUhEUgAAA2oAAAHkCAYAAABPDtyKAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMA'

In [4]:
document = {
    "image_data": image_base64
}

response = es.index(index="my_index", body=document)
print(f"Document {response["_id"]} is {response["result"]} and is split into {response["_shards"]["total"]} shards.")

Document mqZ1OpoBbrc3W2pBl7JS is created and is split into 2 shards.


1.2 Others

In [5]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(
    index="my_index",
    mappings={
        "properties":{
            "book_reference": {
                "type": "keyword"
            },
            "price": {
                "type": "float"
            },
            "publish_date": {
                "type": "date"
            },
            "is_available": {
                "type": "boolean"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [6]:
document = {
    "book_reference": "978-1617294433",
    "price": 44.99,
    "publish_date": "2021-06-30",
    "is_available": True
}

response = es.index(index="my_index", body=document)
print(f"Document {response["_id"]} is {response["result"]} and is split into {response["_shards"]["total"]} shards.")

Document m6Z1OpoBbrc3W2pBn7KT is created and is split into 2 shards.


2. Object types

2.1 Object

In [7]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(
    index='my_index', 
    mappings={
        'properties': {
            'author': {
                'properties': {
                    'first_name': {
                        'type': 'text'
                    },
                    'last_name': {
                        'type': 'text'
                    }
                }
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [8]:
document = {
    'author': {
        'first_name': 'Anamul Islam',
        'last_name': 'Shamim'
    }
}

response = es.index(index='my_index', body=document)
response

ObjectApiResponse({'_index': 'my_index', '_id': 'nKZ1OpoBbrc3W2pBp7JI', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

2.2 Flattened object

In [9]:
es.indices.delete(index='my_index', ignore_unavailable=True) 
es.indices.create(
    index='my_index',
    mappings={
        "properties": {
            "author": {
                "type": "flattened"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [10]:
document = {
    "author": {
        "first_name": {"type": "text"},
        "last_name": {"type": "text"}
    }
}

response = es.index(index="my_index", body=document)
print(f"Document {response["_id"]} is {response["result"]} and is split into {response["_shards"]["total"]} shards.")

Document naZ1OpoBbrc3W2pBrrIJ is created and is split into 2 shards.


2.3 Nested object

In [11]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(
    index="my_index",
    mappings={
        "properties":{
            "user": {
                "type":"nested"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [12]:
documents = [
    {
        "first": "John",
        "last": "Smith"
    },
    {
        "first": "Imad",
        "last": "Saddik"
    }
]

response = es.index(index="my_index", body={"user": documents})
response

ObjectApiResponse({'_index': 'my_index', '_id': 'nqZ1OpoBbrc3W2pBsrLn', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

3. Text search types

3.1 Text

In [13]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(
    index="my_index",
    mappings={
        "properties": {
            "email_body": {
                "type": "text"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

3.2 Completion

In [14]:
document = {
    "email_body": "Hello, this is a test email."
}
response = es.index(index='my_index', body=document)
response

ObjectApiResponse({'_index': 'my_index', '_id': 'n6Z2OpoBbrc3W2pBzLJx', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

3.2 Completion

In [16]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(
    index="my_index", 
    mappings={
        "properties":{
            "suggest": {
                "type": "completion"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [17]:
document_1 = {
    "suggest": {
        "input": ["Mars", "Planet"]
    }
}

document_2 = {
    "suggest": {
        "input": ["Andromida", "Galaxy"]
    }
}

es.index(index="my_index", body=document_1)
es.index(index="my_index", body=document_2)

ObjectApiResponse({'_index': 'my_index', '_id': 'o6Z8OpoBbrc3W2pBY7KS', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

4. Spatial data types

4.1 Geo point

In [18]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(
    index="my_index",
    mappings={
        "properties":{
            "location":{
                "type": "geo_point"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [19]:
document = {
    "text": "Geopoint as an object using GeoJSON format",
    "location": {
        "type": "Point",
        "coordinates": [
            -71.41,
            41.12
        ]
    }
}

response = es.index(index="my_index", body=document)
response

ObjectApiResponse({'_index': 'my_index', '_id': 'pKbFOpoBbrc3W2pBALKP', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

4.2 Geo shape

In [20]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(
    index="my_index",
    mappings={
        "properties": {
            "location": {
                "type": "geo_shape"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [ ]:
document_1 = {
    "location": {
        "type": "LineString",
        "coordinates": [
            [
                -77.03653,
                38.897676
            ],
            [
                -77.009051,
                38.889939
            ]
        ]
    }
}

document_2 = {
    "location": {
        "type": "Polygon",
        "coordinates": [
            [
                [100, 0],
                [101, 0],
                [101, 1],
                [100, 1],
                [100, 0],
            ],
            [
                [100.2, 0.2],
                [100.8, 0.2],
                [100.8, 0.8],
                [100.2, 0.8],
                [100.2, 0.2],
            ]
        ]
    }
}

es.index(index="my_index", body=document_1)
es.index(index="my_index", body=document_2)

ObjectApiResponse({'_index': 'my_index', '_id': 'qKbYOpoBbrc3W2pB07Lc', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

4.3 Point

In [25]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(
    index="my_index",
    mappings={
        "properties": {
            "location": {
                "type": "point"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [26]:
document = {
    "location": {
        "type": "Point", 
        "coordinates": [
            -71.34,
            41.12
        ]
    }
}

es.index(index="my_index", body=document)

ObjectApiResponse({'_index': 'my_index', '_id': 'qab7OpoBbrc3W2pBEbJ1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})